Helper Functions

In [ ]:
import pickle
from google.colab import files
from google.colab.patches import cv2_imshow
import skimage.transform
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data import Dataset
from pathlib import Path
import time
import cv2
import torchvision
from torchvision import datasets, transforms
import torch.nn.functional as F
#import tqdmpi
from scipy.stats import ttest_rel
import pandas as pd
import sklearn, sklearn.metrics
from skimage.io import imread
import torchxrayvision as xrv
import scipy
from scipy import stats
from torch._C import NoneType
import statistics
import seaborn as sns
import copy
from operator import add
from operator import truediv

import os,sys
os.environ['OPENCV_IO_ENABLE_JASPER'] = 'True'

from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

contentFolder = '/content/drive/MyDrive/UM2ii/'

destinationFolder = '/content/'

#Default return from XRV models (except for Official Chexpert model)
default_pathologies = ['Atelectasis',
 'Consolidation',
 'Infiltration',
 'Pneumothorax',
 'Edema',
 'Emphysema',
 'Fibrosis',
 'Effusion',
 'Pneumonia',
 'Pleural_Thickening',
 'Cardiomegaly',
 'Nodule',
 'Mass',
 'Hernia',
 'Lung Lesion',
 'Fracture',
 'Lung Opacity',
 'Enlarged Cardiomediastinum']

#Pathologies we are using
included_pathologies = ['Atelectasis',
 'Pneumothorax',
 'Edema',
 'Effusion',
 'Pneumonia',
 'Cardiomegaly']

#Default return from Official Chexpert model
chex_pathologies = ["Atelectasis", "Cardiomegaly", "Consolidation", "Edema", "Effusion"]

#Preprocessing methods we are using
preprocessingMethods = [
                        "ir_bilinear_skimage",
                        "ir_interarea_cv2",
                        "ir_bilinear_cv2",
                        "ir_cubic_cv2",
                        "ir_lanczos_cv2",
                        "clahe_true_bilinear_skimage",
                        "compr_j2k_bilinear_skimage",
                        "compr_j2k_cr2_bilinear_skimage",
                        "compr_j2k_cr4_bilinear_skimage",
                        "compr_j2k_cr8_bilinear_skimage",
                        "compr_j2k_cr16_bilinear_skimage",
                        "compr_j2k_cr32_bilinear_skimage"
                        ]


#Identify subset of NIH images that are coded as RGB
#RGB_indices list is used to exclude these images during analysis
files = os.listdir('/content/NIH_cohort_balanced/')
files.sort()
RGB_indices = []
for n in range(len(files)):
    #Iterate through sorted files
    #Ensure we are only loading image files
    if 'png' in files[n]:
      Img = Image.open('/content/NIH_cohort_balanced/' + files[n])
      if (Img.mode != "L"):
        RGB_indices.append(n)
print(RGB_indices)
print(len(RGB_indices))


def auc(labels,outputs):
   """
   Finds AUROC given labels and outputs, assumes included_pathologies is defined globally

    Args:
        labels: list of lists of labels
        outputs: list of lists of outputs
    Returns:
        Prints AUROC for each pathology

   """
   for i in range(6):
    if len(np.unique(np.asarray(labels)[:,i])) > 1:
        auc = sklearn.metrics.roc_auc_score(np.asarray(labels)[:,i], np.asarray(outputs)[:,i])
    else:
        auc = "(Only one class observed)"
    print(included_pathologies[i], auc)



def aucChex(labels,outputs):
  """
  Does the same thing as auc() but for the official Chexpert model, utilizing chex_pathologies instead of included_pathologies

  Args:
      labels: list of lists of labels
      outputs: list of lists of outputs
  Returns:
      None
  """

  for i in range(len(chex_pathologies)):
    if len(np.unique(np.asarray(labels)[:,i])) > 1:
        labelsIndex = None
        MatchIdentified = False
        #Need to figure out which labels correspond to this chex model output
        for p in range(len(included_pathologies)):
          if str(included_pathologies[p]) == str(chex_pathologies[i]):
            labelsIndex = p
            MatchIdentified = True
        if MatchIdentified:
          auc = sklearn.metrics.roc_auc_score(np.asarray(labels)[:,labelsIndex], np.asarray(outputs)[:,i])
          print(chex_pathologies[i], auc)
    else:
        auc = "(Only one class observed)"

class customXRayResizer(object):

  """
  Over-writing the default XRV resizer to use alternate methods of resizing
  Return resized image based on engine and resizing method input on call

  """
  def __init__(self, size, engine="cv2", interp="interarea"):
      self.size = size
      self.engine = engine
      self.interp = interp
  def __call__(self, img):
      if(self.engine == 'skimage'):
        return skimage.transform.resize(img, (1, self.size, self.size), mode='constant', preserve_range=True).astype(np.float32)
      elif (self.engine == 'cv2'):
        if (self.interp == 'interarea'):
          return cv2.resize(img[0,:,:],
                              (self.size, self.size),
                              interpolation = cv2.INTER_AREA
                            ).reshape(1,self.size,self.size).astype(np.float32)
        elif(self.interp == 'bilinear'):
          return cv2.resize(img[0,:,:],
                              (self.size, self.size),
                              interpolation = cv2.INTER_LINEAR
                            ).reshape(1,self.size,self.size).astype(np.float32)
        elif(self.interp == 'nearest'):
          return cv2.resize(img[0,:,:],
                              (self.size, self.size),
                              interpolation = cv2.INTER_NEAREST
                            ).reshape(1,self.size,self.size).astype(np.float32)
        elif(self.interp == 'cubic'):
          return cv2.resize(img[0,:,:],
                              (self.size, self.size),
                              interpolation = cv2.INTER_CUBIC
                            ).reshape(1,self.size,self.size).astype(np.float32)
        elif(self.interp == 'lanczos'):
          return cv2.resize(img[0,:,:],
                              (self.size, self.size),
                              interpolation = cv2.INTER_LANCZOS4
                            ).reshape(1,self.size,self.size).astype(np.float32)

def apply_model(model, clahe = False, resize = 'bilinear', eng = 'cv2', fileDirectory="", fileNames=[]):

  """
  Applies model to images in fileDirectory using fileNames, returns list of model predictions

  Args:
      model: model to apply
      clahe: boolean, whether to apply CLAHE to images
      resize: string, resizing method to use
      eng: string, engine to use for resizing
      fileDirectory: string, directory containing images to apply model to
      fileNames: list of strings, names of files to apply model to
  Returns:
      modelPredictions: list of lists of model predictions
  """

  modelPredictions = []

  model.op_threshs = None
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  model.to(device)

  with torch.no_grad():
      #Iterate through filenames and process each image
      for i in range(len(fileNames)):

          #Read next image from file and apply CLAHE if requested
          if(clahe):
            #Reading the image from the next image directory
            tmpImg = cv2.imread(fileDirectory + fileNames[i])
            #Ensure black and white color scheme
            image_bw = cv2.cvtColor(tmpImg, cv2.COLOR_BGR2GRAY)

            # The declaration of CLAHE
            clahe = cv2.createCLAHE()
            tmpImg = clahe.apply(image_bw)

          else:
            #No CLAHE -> read image using default skimage.imread
            tmpImg = imread(fileDirectory + fileNames[i])
            #Ensure black and white color scheme
            #tmpImg = cv2.cvtColor(tmpImg, cv2.COLOR_BGR2GRAY)


          #Normalize image
          tmpImg = xrv.datasets.normalize(tmpImg, 255)

          # Check that images are 2D arrays
          if len(tmpImg.shape) > 2:
              tmpImg = tmpImg[:, :, 0]
          if len(tmpImg.shape) < 2:
              print("error, dimension lower than 2 for image")

          # Add color channel
          tmpImg = tmpImg[None, :, :]

          #RESIZE
          #Get custom resizer indicating engine and resizing method
          transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),
                                                  customXRayResizer(size=224, engine=eng, interp=resize)])

          #Apply resizing
          tmpImg = transform(tmpImg)

          #Get model predictions
          currentPrediction = model(torch.from_numpy(tmpImg).to(device).unsqueeze(0))
          modelPredictions.append(currentPrediction.detach().cpu().numpy()[0])
          tmpImg = torch.from_numpy(tmpImg).to(device).unsqueeze(0)
          #print(i)
          if i % 100 == 0:
            print(f"{i} images done")
      return modelPredictions

def organizeImages(fileDir="", labelsDir="", fileExtension=""):

  """
  Organizes images in fileDir and labels in labelsDir into two lists, one containing image names and one containing labels
  These are returned in the same case order

  Args:
      fileDir: string, directory containing images
      labelsDir: string, directory containing labels
      fileExtension: string, file extension of images
  Returns:
      imgNames: list of strings, names of images in fileDir
      labels: list of strings, labels of images in fileDir in same order as imgNames
  """
#Return two variables; containing image names and disease labels in the same order

  #Read labels file
  labcsv = pd.read_csv(labelsDir)
  #List of image names
  series = labcsv['Image']

  updatedNames = []

  #Add each image file to list of names without file extension
  for x in range(len(series)):
    if(series[x].endswith('.png') or series[x].endswith('.jpg')):
      updatedNames.append(series[x][:-4]+fileExtension)

  #Convert to series
  updatedNames = pd.Series(updatedNames)

  #List files
  files = os.listdir(fileDir)
  files.sort()

  #Create new lists that will contain sorted labels and image names
  labels = []
  imgNames = []
  #For each image, identify the corresponding line in the labels file.
  #Add the corresponding label to labels[]

  for file in files:
    #Iterate through sorted files
    #Ensure we are only loading image files
    if fileExtension in file:

      #Get index of current file in labels series
      index = updatedNames[updatedNames == file].index[0]
      #Get disease labels data for current file based on index
      nextLab = labcsv.iloc[index].to_numpy()[1:]

      #Add current file labels to ordered list of labels
      labels.append(nextLab)
      #Add current file name to ordered list of image names
      imgNames.append(file)

  #Convert to np array
  labels = np.array(labels, dtype=np.float32)

  return imgNames,labels

def generateTable(pathologyIndex=0,sm=[]):

  """
  Generates table of values for a given pathology index

  Args:
      pathologyIndex: int, index of pathology to generate table for
      sm: list of lists of model predictions
  Returns:
      vals: list of floats, values for pathologyIndex
  """
  #Reformat data to generate table
  vals = []
  for i in range(len(sm)):
    vals.append(sm[i][pathologyIndex])
  return vals

def pickleVar(path="", var=""):
  """
  Pickle/Save a variable as a file

  Args:
      path: string, path to save file to
      var: variable to save
  Returns:
      None

  """
  with open(path, "wb") as fp:   #Pickling
    pickle.dump(var, fp)

def getImageDimensions(dir):
  """
  Iterates through images in a directory and prints metrics of image dimensions

  Args:
      dir: string, directory to iterate through
  Returns:
      None
  """

  #Prints metrics of image dimensions for a given directory
  parentDir = dir

  widths = []
  heights = []
  areas = []
  size = []
  bitmaps = []

  files = os.listdir(parentDir)
  files.sort()

  #Iterate through files and collect stats
  for x in range(len(files)):
      #Ensure we are only examining image files
      if '.jpg' in str(files[x]) or '.png' in str(files[x]) or '.j2k' in str(files[x]):
          currentPath = parentDir + files[x]
          file_stats = os.stat(currentPath)
          size.append(file_stats.st_size)

          img = Image.open(currentPath)
          widths.append(img.width)
          heights.append(img.height)
          areas.append(img.width*img.height)
          bitmaps.append(img.width*img.height*8)


  print("Num images:")
  print(len(widths))

  print("Avg Uncompressed Bitmap Kb")
  print((np.sum(bitmaps)/len(bitmaps))/(8000) )
  print(f"({str(np.std( np.divide(bitmaps, 8000) ) )})")

  print("Sum Uncompressed Bitmap Kb")
  print((np.sum(bitmaps))/1000 )

  print("AVERAGE (STDEV) WIDTH:")
  print(np.sum(widths)/len(widths))
  print(f"({str(np.std(widths))})")

  print("Width Min, Max")
  print(min(widths))
  print(max(widths))

  print("AVERAGE (STDEV) HEIGHT:")
  print(np.sum(heights)/len(heights))
  print(f"({str(np.std(heights))})")

  print("Height Min, Max")
  print(min(heights))
  print(max(heights))

  print("AVERAGE (STDEV) AREA:")
  print(np.sum(areas)/len(areas))
  print(f"({str(np.std(areas))})")

  print("Area Min, Max")
  print(min(areas))
  print(max(areas))

  print("Sum size, Kb")
  print((np.sum(size))/1000 )

  print("AVERAGE (STDEV) SIZE bytes:")
  print(np.sum(size)/len(size))
  print(f"({str(np.std(size))})")

  print("AVERAGE (STDEV) SIZE IN MB")
  print((np.sum(size)/len(size))/1000000)
  print(f"({str(np.std(np.divide(size, 1000000)))})")

def loadSavedVariables(pathologies, model, dataset, preprocessingMethods):
  """
  Loads saved variables (previously pickled predictions) based on a model, dataset, and preprocessing methods.
  Assumes variables are saved in the AutoSavedVariables folder under naming conventions of model_dataset_preprocessingMethod

  Args:
      pathologies: list of strings, list of pathologies to load variables for
      model: string, model to load variables for
      dataset: string, dataset to load variables for
      preprocessingMethods: list of strings, list of preprocessing methods to load variables for

  Returns:
      results_list: list of lists of floats, list of raw predictions for each preprocessing method
      frames: dictionary of dataframes, dictionary of dataframes of predictions organized by pathology
  """

  #Load saved variables based on a model, dataset, and preprocessing methods
  #Returns list of raw predictions and frame reorganized by disease label for AUC calculation
  results_list = []

  #For each preprocessing method, attempt to load the saved model
  for t in preprocessingMethods:
    currentPath = contentFolder + "AutoSavedVariables/" + model + "_" + dataset + "_" + t
    if os.path.isfile(currentPath):
      with open(currentPath, "rb") as fp:   # Unpickling
        currentVar = pickle.load(fp)
      results_list.append(currentVar)
    else:
      print(currentPath + " does not exist")

  frames = {}
  for x in range(len(pathologies)):
    df = pd.DataFrame()
    for y in range(len(results_list)):
      df[str(y)] = pd.Series(generateTable(x, results_list[y]))
    frames[pathologies[x]] = df

  return results_list, frames



def get_base_comparison_labels_and_preds(conditionIndex, variables_jpg, base, basename,
                                 comparison, comparisonname, isChexModel):

  """
  Minor function to pull specific prediction data, creates a dataframe based on loaded data, a given pathology index, and base and comparison groups (two types of preprocessing).


  Args:
    conditionIndex: index of pathology in variables_jpg
    variables_jpg: loaded prediction outputs from the previous results
    base: list of predictions from base preprocessing
    basename: name of base preprocessing
    comparison: list of predictions from comparison preprocessing
    comparisonname: name of comparison preprocessing
    isChexModel: boolean to indicate if we are using chexmodel

  Returns:
    roc_df: dataframe with columns:
      y_test_conditionName_basename representing true labels
      preds_conditionName_basename representing predictions from base preprocessing
      preds_conditionName_comparisonname representing predictions from comparison preprocessing

  """

  #Create dataframe for export
  roc_df = pd.DataFrame()

  varsjpgIndex = conditionIndex

  #If we are using chexmodel, ensure we have correct pathology matched up since it returns in a different order
  if (isChexModel):
    for x in range(len(included_pathologies)):
      if (chex_pathologies[conditionIndex] == included_pathologies[x]):
        varsjpgIndex = x
  conditionName = included_pathologies[varsjpgIndex]


  y_test = np.asarray(variables_jpg[1])[:,varsjpgIndex]
  preds = np.asarray(base)[:,conditionIndex]

  roc_df['y_test_' + conditionName + '_' + basename] = pd.Series(y_test)
  roc_df['preds_' + conditionName + '_' + basename] = pd.Series(preds)

  preds = np.asarray(comparison)[:,conditionIndex]

  roc_df['preds_' + conditionName + '_' + comparisonname] = pd.Series(preds)

  return roc_df

def get_predictions_and_scatterpoints(model_results_name, dataset_results_name, conditionIndex, doCLAHEandJ2K, doCompRatio, savedVariables):

  """
  Main function to obtain raw predictions and normalized predictions for scatterplots, organized into two dataframes.

  Args:
        model_results_name: name of model results to use, models = ['chexmodel', 'xrv_nih', 'xrv_mimic', 'xrv_chex']
        dataset_results_name: name of dataset results to use, datasets = ['chex', 'mimic', 'NIH']
        conditionIndex: index of condition to use, conditions = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']
        doCLAHEandJ2K: boolean to indicate whether to include CLAHE and J2K in the results
        doCompRatio: boolean to indicate whether to include compression ratios in the results
        savedVariables: loaded prediction outputs from the previous results

  Returns:
        condition_df: dataframe containing the true labels, and raw predictions for each preprocessing type organized as columns
        scatter_df: dataframe formatted to plot (x preds and y preds for each test). The x all represent preds from standard processing,
    but are normalized relative to the minimum and maximum prediction values of that y pred preprocessing type.

  """


  condition_df = pd.DataFrame()
  scatter_df = pd.DataFrame()

  if model_results_name == 'chexmodel':
    condition = chex_pathologies[conditionIndex]
    pathologies = chex_pathologies

    chexModel = True
  else:
    condition = included_pathologies[conditionIndex]
    pathologies = included_pathologies
    chexModel = False


  irNames = ['Bilinear',
          'Interarea',
          'Bilinear CV2',
          'Cubic',
          'Lanczos']

  crNames = ['2', '4', '8', '16', '32']



  #IR
  #iterate through different types of resizing
  for x in range(len(irNames)):

    #Normalize standard and resizing data relative to the resizing prediction bounds
    mi = min(min(savedVariables[1][condition].iloc[:,0]), min(savedVariables[1][condition].iloc[:,x]))
    ma = max(max(savedVariables[1][condition].iloc[:,0]), max(savedVariables[1][condition].iloc[:,x]))

    x_ax = (savedVariables[1][condition].iloc[:,0] - mi) / (ma - mi)
    y_ax = (savedVariables[1][condition].iloc[:,x] - mi) / (ma - mi)

    scatter_df[irNames[x] + '_x'] = x_ax
    scatter_df[irNames[x] + '_y'] = y_ax

    #On first iteration, create a new dataframe with results
    if (condition_df.empty):

      condition_df = pd.DataFrame(get_base_comparison_labels_and_preds(
                            conditionIndex, vars_jpg,
                            savedVariables[0][0],
                            irNames[0],
                            savedVariables[0][x],
                            irNames[x],
                            chexModel))

    #Otherwise, concatenate the new predictions only to existing df
    else:

      nf = pd.DataFrame(get_base_comparison_labels_and_preds(conditionIndex, vars_jpg,
                              savedVariables[0][0],
                              irNames[0],
                              savedVariables[0][x],
                              irNames[x],
                              chexModel))


      condition_df = pd.concat([condition_df, nf.iloc[:,-1]], axis=1)



  if doCLAHEandJ2K:

    #Clahe
    #zi = (xi – min(x)) / (max(x) – min(x))
    mi = min(min(savedVariables[1][condition].iloc[:,0]), min(savedVariables[1][condition].iloc[:,5]))
    ma = max(max(savedVariables[1][condition].iloc[:,0]), max(savedVariables[1][condition].iloc[:,5]))

    x_ax = (savedVariables[1][condition].iloc[:,0] - mi) / (ma - mi)
    y_ax = (savedVariables[1][condition].iloc[:,5] - mi) / (ma - mi)

    scatter_df['CLAHE' + '_x'] = x_ax
    scatter_df['CLAHE' + '_y'] = y_ax


    nf = pd.DataFrame(get_base_comparison_labels_and_preds(conditionIndex, vars_jpg,
                              savedVariables[0][0],
                              irNames[0],
                              savedVariables[0][5],
                              'CLAHE TRUE',
                              chexModel))
    condition_df = pd.concat([condition_df, nf.iloc[:,-1]], axis=1)



    #J2k
    mi = min(min(savedVariables[1][condition].iloc[:,0]), min(savedVariables[1][condition].iloc[:,6]))
    ma = max(max(savedVariables[1][condition].iloc[:,0]), max(savedVariables[1][condition].iloc[:,6]))

    x_ax = (savedVariables[1][condition].iloc[:,0] - mi) / (ma - mi)
    y_ax = (savedVariables[1][condition].iloc[:,6] - mi) / (ma - mi)

    scatter_df['J2K' + '_x'] = x_ax
    scatter_df['J2K' + '_y'] = y_ax


    nf = get_base_comparison_labels_and_preds(conditionIndex, vars_jpg,
                              savedVariables[0][0],
                              irNames[0],
                              savedVariables[0][6],
                              'J2K',
                              chexModel)

    condition_df = pd.concat([condition_df, nf.iloc[:,-1]], axis=1)



  if doCompRatio:
    for x in range(7, 12):

      mi = min(min(savedVariables[1][condition].iloc[:,0]), min(savedVariables[1][condition].iloc[:,x]))
      ma = max(max(savedVariables[1][condition].iloc[:,0]), max(savedVariables[1][condition].iloc[:,x]))

      x_ax = (savedVariables[1][condition].iloc[:,0] - mi) / (ma - mi)
      y_ax = (savedVariables[1][condition].iloc[:,x] - mi) / (ma - mi)

      scatter_df[crNames[x-7] + '_x'] = x_ax
      scatter_df[crNames[x-7] + '_y'] = y_ax

      nf =  get_base_comparison_labels_and_preds(conditionIndex, vars_jpg,
                              savedVariables[0][0],
                              irNames[0],
                              savedVariables[0][x],
                              crNames[x-7],
                              chexModel)
      condition_df = pd.concat([condition_df, nf.iloc[:,-1]], axis=1)

  return condition_df, scatter_df


def loadModel(modelName):
  if modelName == "xrv_nih":
    model = xrv.models.DenseNet(weights="densenet121-res224-nih", apply_sigmoid = False)
  elif modelName == "xrv_chex":
    model = xrv.models.DenseNet(weights="densenet121-res224-chex", apply_sigmoid = False)
  elif modelName == "xrv_mimic":
    model = xrv.models.DenseNet(weights="densenet121-res224-mimic_ch", apply_sigmoid = False)
  elif modelName == "chexmodel":
    # Official Stanford CheXpert model
    model = xrv.baseline_models.chexpert.DenseNet(weights_zip=contentFolder + 'chexpert_weights.zip', num_models=30)
  return model

def getYoudensThreshold(y_true, y_score):
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(y_true, y_score)
    idx = np.argmax(tpr - fpr)
    return thresholds[idx]

def getCounts(df, thresholdOriginal):


  df['res_Bilinear'] = ""


  #Sensitivity = True positive / (true positive + false negatives)
  #Specificity = True negative / (true negative + False positive)
  tp_bl = 0
  tn_bl = 0
  fn_bl = 0
  fp_bl = 0

  for i in range(len(df)):
    if (df[df.columns[1]][i] < thresholdOriginal and df[df.columns[0]][i] == 0):
      #True negative
      tn_bl = tn_bl + 1
      df['res_Bilinear'][i] = 'Correct'
    elif (df[df.columns[1]][i] >= thresholdOriginal and df[df.columns[0]][i] == 1):
      #True positive
      tp_bl = tp_bl + 1
      df['res_Bilinear'][i] = 'Correct'
    elif (df[df.columns[1]][i] >= thresholdOriginal and df[df.columns[0]][i] == 0):
      #Predicted as positive but result was negative
      #False positive
      fp_bl = fp_bl + 1
      df['res_Bilinear'][i] = 'Incorrect'
    elif (df[df.columns[1]][i] < thresholdOriginal and df[df.columns[0]][i] == 1):
      #Predicted as negative but result was positive
      #False negative
      fn_bl = fn_bl + 1
      df['res_Bilinear'][i] = 'Incorrect'

  # Sensitivity
  if (tp_bl + fn_bl) != 0:
      sens_bl = tp_bl / (tp_bl + fn_bl)
  else:
      sens_bl = 0

  # Specificity
  if (tn_bl + fp_bl) != 0:
      spec_bl = tn_bl / (tn_bl + fp_bl)
  else:
      spec_bl = 0

  # Positive Predictive Value (PPV) or Precision
  if (tp_bl + fp_bl) != 0:
      ppv_bl = tp_bl / (tp_bl + fp_bl)
  else:
      ppv_bl = 0

  # Negative Predictive Value (NPV)
  if (tn_bl + fn_bl) != 0:
      npv_bl = tn_bl / (tn_bl + fn_bl)
  else:
      npv_bl = 0

  # Accuracy
  total_bl = tp_bl + tn_bl + fp_bl + fn_bl
  if total_bl != 0:
      accuracy_bl = (tp_bl + tn_bl) / total_bl
  else:
      accuracy_bl = 0



  res_names = ["res_bilinear_full", "res_Interarea", "res_BilinearCV2", "res_Cubic", "res_Lanczos", "res_CLAHE", "res_J2K", "res_2", "res_4", "res_8", "res_16", "res_32"]
  pred_names = df.columns[1:]

  tp_proc = [0] * len(res_names)
  tn_proc = [0] * len(res_names)
  fp_proc = [0] * len(res_names)
  fn_proc = [0] * len(res_names)

  AUROC_proc = [0] * len(res_names)


  for i in range(len(res_names)):

    #AUROC for each form of preprocessing
    #auc = sklearn.metrics.roc_auc_score(np.asarray(labels)[:,i], np.asarray(outputs)[:,i])
    y_test = df[df.columns[0]][:]
    y_pred = df[df.columns[i+1]][:] #1 = bilinear, 2 = interarea...
    AUROC_proc[i] = sklearn.metrics.roc_auc_score(np.asarray(y_test), np.asarray(y_pred))

    #y_test = () #true labels
    #print(df[df.columns[1]][:])

    df[res_names[i]] = ""
    for l in range(len(df)):
      if( df[pred_names[i]][l] < thresholdOriginal and df[df.columns[0]][l] == 0 ):
        #True negative
        tn_proc[i] = tn_proc[i] + 1
        if(df['res_Bilinear'][l] == 'Correct'):
          df[res_names[i]][l] = 'Consistently Correct'
        else:
          df[res_names[i]][l] = 'Wrong Original, Correct Transformed'
      elif( df[pred_names[i]][l] >= thresholdOriginal and df[df.columns[0]][l] == 1 ):
        #True positive
        tp_proc[i] = tp_proc[i] + 1
        if(df['res_Bilinear'][l] == 'Correct'):
          df[res_names[i]][l] = 'Consistently Correct'
        else:
          df[res_names[i]][l] = 'Wrong Original, Correct Transformed'
      elif( df[pred_names[i]][l] >= thresholdOriginal and df[df.columns[0]][l] == 0 ):
        #Predicted as positive, true answer was negative
        #False positive
        fp_proc[i] = fp_proc[i] + 1
        if(df['res_Bilinear'][l] == 'Correct'):
          df[res_names[i]][l] = 'Correct Original, Wrong Transformed'
        else:
          df[res_names[i]][l] = 'Consistently Wrong'
      elif( df[pred_names[i]][l] < thresholdOriginal and df[df.columns[0]][l] == 1 ):
        #Predicted as negative, true answer was positive
        #False negative
        fn_proc[i] = fn_proc[i] + 1
        if(df['res_Bilinear'][l] == 'Correct'):
          df[res_names[i]][l] = 'Correct Original, Wrong Transformed'
        else:
          df[res_names[i]][l] = 'Consistently Wrong'
      else:
        print("Something went wrong")




  # Convert the lists to numpy arrays for element-wise addition
  tp_np = np.array(tp_proc)
  tn_np = np.array(tn_proc)
  fp_np = np.array(fp_proc)
  fn_np = np.array(fn_proc)

  # Sensitivity of preprocessed
  sens_proc_num = tp_np
  sens_proc_num = np.nan_to_num(sens_proc_num, nan=0)
  sens_proc_denom = np.add(tp_np, fn_np)
  sens_proc_denom = np.nan_to_num(sens_proc_denom, nan=0)
  sens_proc = np.divide(sens_proc_num, sens_proc_denom)
  sens_proc = np.nan_to_num(sens_proc, nan=0)
  # Iterate through the elements of sens_proc_num and sens_proc_denom
  sens_proc_frac = []
  for num, denom in zip(sens_proc_num, sens_proc_denom):
      # Construct the fraction string
      fraction_str = f"{num}/{denom}"

      # Append the fraction string to the sens_proc_frac list
      sens_proc_frac.append(fraction_str)


  # Specificity of preprocessed
  spec_proc_num = tn_np
  spec_proc_num = np.nan_to_num(spec_proc_num, nan=0)
  spec_proc_denom = np.add(tn_np, fp_np)
  spec_proc_denom = np.nan_to_num(spec_proc_denom, nan=0)
  spec_proc = np.divide(spec_proc_num, spec_proc_denom)
  spec_proc = np.nan_to_num(spec_proc, nan=0)

  # Calculate Positive Predictive Value (PPV) or Precision
  ppv_proc_num = tp_np
  ppv_proc_num = np.nan_to_num(ppv_proc_num, nan=0)
  ppv_proc_denom = np.add(tp_np, fp_np)
  ppv_proc_denom = np.nan_to_num(ppv_proc_denom, nan=0)
  ppv_proc = np.divide(ppv_proc_num, ppv_proc_denom)
  ppv_proc = np.nan_to_num(ppv_proc, nan=0)


  # Negative Predictive Value (NPV)
  npv_proc_num = tn_np
  npv_proc_num = np.nan_to_num(npv_proc_num, nan=0)
  npv_proc_denom = np.add(tn_np, fn_np)
  npv_proc_denom = np.nan_to_num(npv_proc_denom, nan=0)
  npv_proc = np.divide(npv_proc_num, npv_proc_denom)
  npv_proc = np.nan_to_num(npv_proc, nan=0)


  # Accuracy
  accuracy_proc_num = np.add(tp_np, tn_np)
  accuracy_proc_num = np.nan_to_num(accuracy_proc_num, nan=0)
  accuracy_proc_denom = np.add(np.add(tp_np, tn_np), np.add(fp_np, fn_np))
  accuracy_proc_denom = np.nan_to_num(accuracy_proc_denom, nan=0)
  accuracy_proc = np.divide(accuracy_proc_num, accuracy_proc_denom)
  accuracy_proc = np.nan_to_num(accuracy_proc, nan=0)

  percent_df = pd.DataFrame(columns = [

    "BaselineCorrect",
    "Consistently Correct",
    "Consistently Wrong",
    "Correct Original, Wrong Transformed",
    "Wrong Original, Correct Transformed",
    "PercentFlipped",
    "PreprocessedSensitivity",
    "PreprocessedSpecificity",
    "PreprocessedPPV",
    "PreprocessedNPV",
    "PreprocessedAccuracy",
    "PreprocessedAUROC"

  ])
  count_df = pd.DataFrame(columns = [

    "BaselineCorrect",
    "Consistently Correct",
    "Consistently Wrong",
    "Correct Original, Wrong Transformed",
    "Wrong Original, Correct Transformed",
    "PercentFlipped",
    "BaselineSensitivity",
    "BaselineSpecificity",
    "PreprocessedSensitivity",
    "PreprocessedSpecificity"
  ])
  #print(df)
  for x in range(len(res_names)):
    percent = (df[res_names[x]].value_counts(normalize=True) * 100)
    percent = percent.rename(percent.name + "_test")
    percent_df = percent_df.append(percent)
    percent_df = percent_df.fillna(0)

    count = df[res_names[x]].value_counts(normalize=False)
    count = count.rename(count.name + "_test")
    count_df = count_df.append(count)
    count_df = count_df.fillna(0)


  if('Correct Original, Wrong Transformed' in percent_df and 'Wrong Original, Correct Transformed' in percent_df):

    percent_df['PercentFlipped'] = percent_df['Correct Original, Wrong Transformed'] + percent_df['Wrong Original, Correct Transformed']
    percent_df['BaselineCorrect'] = percent_df['Consistently Correct'] + percent_df['Correct Original, Wrong Transformed']
    percent_df['PreprocessedSensitivity'] = sens_proc
    percent_df['PreprocessedSpecificity'] = spec_proc
    percent_df["PreprocessedPPV"] = ppv_proc
    percent_df["PreprocessedNPV"] = npv_proc
    percent_df["PreprocessedAccuracy"] = accuracy_proc
    percent_df["PreprocessedAUROC"] = AUROC_proc




    count_df['PercentFlipped'] = count_df['Correct Original, Wrong Transformed'] + count_df['Wrong Original, Correct Transformed']
    count_df['BaselineCorrect'] = count_df['Consistently Correct'] + count_df['Correct Original, Wrong Transformed']
    count_df['PreprocessedSensitivity_num'] = sens_proc_num
    count_df['PreprocessedSensitivity_denom'] = sens_proc_denom
    count_df['PreprocessedSpecificity_num'] = spec_proc_num
    count_df['PreprocessedSpecificity_denom'] = spec_proc_denom
    count_df['PreprocessedPPV_num'] = ppv_proc_num
    count_df['PreprocessedPPV_denom'] = ppv_proc_denom
    count_df['PreprocessedNPV_num'] = npv_proc_num
    count_df['PreprocessedNPV_denom'] = npv_proc_denom
    count_df['PreprocessedAccuracy_num'] = accuracy_proc_num
    count_df['PreprocessedAccuracy_denom'] = accuracy_proc_denom


    #count_df['PreprocessedSpecificity'] = spec_proc
  return percent_df, count_df

def getStatisticalResults(pathologies, savedVariables, variables_jpg):
  data = []

  for p in range(len(preprocessingMethods)):
    for x in pathologies:
      l = []
      tval, pval = stats.ttest_rel(savedVariables[1][x].iloc[:,0], savedVariables[1][x].iloc[:,p])
      l.append(f"{preprocessingMethods[p]}_{x}")
      l.append(tval)
      l.append(pval)
      data.append(l)

  df = pd.DataFrame(data, columns=['name', 'T_statistic', 'p_value'])
  return df


def getClassification(df, thresholdOriginal):

  df['res_Bilinear'] = ""

  for i in range(len(df)):
    if (df[df.columns[1]][i] < thresholdOriginal and df[df.columns[0]][i] == 0):
      #True negative
      df['res_Bilinear'][i] = 'Correct'
    elif (df[df.columns[1]][i] >= thresholdOriginal and df[df.columns[0]][i] == 1):
      #True positive
      df['res_Bilinear'][i] = 'Correct'
    elif (df[df.columns[1]][i] >= thresholdOriginal and df[df.columns[0]][i] == 0):
      #Predicted as positive but result was negative
      #False positive
      df['res_Bilinear'][i] = 'Incorrect'
    elif (df[df.columns[1]][i] < thresholdOriginal and df[df.columns[0]][i] == 1):
      #Predicted as negative but result was positive
      #False negative
      df['res_Bilinear'][i] = 'Incorrect'


  res_names = ["res_Interarea", "res_BilinearCV2", "res_Cubic", "res_Lanczos", "res_CLAHE", "res_J2K", "res_2", "res_4", "res_8", "res_16", "res_32"]
  pred_names = df.columns[2:]

  for i in range(len(res_names)):
    df[res_names[i]] = ""
    for l in range(len(df)):
      if( df[pred_names[i]][l] <= thresholdOriginal and df[df.columns[0]][l] == 0 ):
        if(df['res_Bilinear'][l] == 'Correct'):
          df[res_names[i]][l] = 'Consistently Correct'
        else:
          df[res_names[i]][l] = 'Wrong Original, Correct Transformed'
      elif( df[pred_names[i]][l] >= thresholdOriginal and df[df.columns[0]][l] == 1 ):
        if(df['res_Bilinear'][l] == 'Correct'):
          df[res_names[i]][l] = 'Consistently Correct'
        else:
          df[res_names[i]][l] = 'Wrong Original, Correct Transformed'
      else:
        #Transformed was incorrect
        if(df['res_Bilinear'][l] == 'Correct'):
          df[res_names[i]][l] = 'Correct Original, Wrong Transformed'
        else:
          df[res_names[i]][l] = 'Consistently Wrong'
  return df



Load Designated Model

In [ ]:
contentFolder = '/content/Compression/'

destinationFolder = '/content/Compression/'

#Models: "chexmodel" "xrv_nih" "mimic_ch" "xrv_chex"
modelName = "xrv_chex"
model = loadModel(modelName)


Testing By Dataset

In [ ]:
#TEST CHEXPERT DATA

#LOAD LABELS AND IMAGE NAMES
labelsDir = destinationFolder + 'chex_cohort/labels.csv'
fileDir_jpg = destinationFolder + 'chex_cohort/'
vars_jpg = organizeImages(fileDir_jpg, labelsDir, '.jpg')

fileDir_j2k = destinationFolder + 'chex_j2k/'
fileDir_j2k_cr2 = destinationFolder + 'chex_j2k_cr_2/'
fileDir_j2k_cr4 = destinationFolder + 'chex_j2k_cr_4/'
fileDir_j2k_cr8 = destinationFolder + 'chex_j2k_cr_8/'
fileDir_j2k_cr16 = destinationFolder + 'chex_j2k_cr_16/'
fileDir_j2k_cr32 = destinationFolder + 'chex_j2k_cr32/'


vars_j2k = organizeImages(fileDir_j2k, labelsDir, '.j2k')

chex_ir_bilinear_skimage = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_jpg, fileNames=vars_jpg[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_chex_ir_bilinear_skimage",
          var = chex_ir_bilinear_skimage)
chex_ir_interarea_cv2 = apply_model(model,
                    clahe=False, resize = 'interarea', eng = 'cv2', fileDirectory=fileDir_jpg, fileNames=vars_jpg[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_chex_ir_interarea_cv2",
          var = chex_ir_interarea_cv2)
chex_ir_bilinear_cv2 = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'cv2', fileDirectory=fileDir_jpg, fileNames=vars_jpg[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_chex_ir_bilinear_cv2",
          var = chex_ir_bilinear_cv2)
chex_ir_cubic_cv2 = apply_model(model,
                    clahe=False, resize = 'cubic', eng = 'cv2', fileDirectory=fileDir_jpg, fileNames=vars_jpg[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_chex_ir_cubic_cv2",
          var = chex_ir_cubic_cv2)
chex_ir_lanczos_cv2 = apply_model(model,
                    clahe=False, resize = 'lanczos', eng = 'cv2', fileDirectory=fileDir_jpg, fileNames=vars_jpg[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_chex_ir_lanczos_cv2",
          var = chex_ir_lanczos_cv2)

#Test CLAHE
chex_clahe_true_bilinear_skimage = apply_model(model,
                      clahe=True, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_jpg, fileNames=vars_jpg[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_chex_clahe_true_bilinear_skimage",
            var = chex_clahe_true_bilinear_skimage)
#Test Compression
chex_compr_j2k_bilinear_skimage = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_j2k, fileNames=vars_j2k[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_chex_compr_j2k_bilinear_skimage",
          var = chex_compr_j2k_bilinear_skimage)


#Compression by compression ratio

chex_compr_j2k_cr2_bilinear_skimage = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_j2k_cr2, fileNames=vars_j2k[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_" + "chex_compr_j2k_cr2_bilinear_skimage",
          var = chex_compr_j2k_cr2_bilinear_skimage)
chex_compr_j2k_cr4_bilinear_skimage = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_j2k_cr4, fileNames=vars_j2k[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_" + "chex_compr_j2k_cr4_bilinear_skimage",
          var = chex_compr_j2k_cr4_bilinear_skimage)
chex_compr_j2k_cr8_bilinear_skimage = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_j2k_cr8, fileNames=vars_j2k[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_" + "chex_compr_j2k_cr8_bilinear_skimage",
          var = chex_compr_j2k_cr8_bilinear_skimage)
chex_compr_j2k_cr16_bilinear_skimage = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_j2k_cr16, fileNames=vars_j2k[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_" + "chex_compr_j2k_cr16_bilinear_skimage",
          var = chex_compr_j2k_cr16_bilinear_skimage)
chex_compr_j2k_cr32_bilinear_skimage = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_j2k_cr32, fileNames=vars_j2k[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_" + "chex_compr_j2k_cr32_bilinear_skimage",
          var = chex_compr_j2k_cr32_bilinear_skimage)

#TEST NIH DATA
#LOAD LABELS AND IMAGE NAMES
labelsDir = destinationFolder + 'NIH_cohort_balanced/labels.csv'
fileDir_jpg = destinationFolder + 'NIH_cohort_balanced/'
vars_jpg = organizeImages(fileDir_jpg, labelsDir, '.png')

fileDir_j2k = destinationFolder + 'NIH_J2K_balanced/'
fileDir_j2k_cr2 = destinationFolder + 'NIH_j2k_cr_2/'
fileDir_j2k_cr4 = destinationFolder + 'NIH_j2k_cr_4/'
fileDir_j2k_cr8 = destinationFolder + 'NIH_j2k_cr_8/'
fileDir_j2k_cr16 = destinationFolder + 'NIH_j2k_cr_16/'
fileDir_j2k_cr32 = destinationFolder + 'NIH_j2k_cr32/'


vars_j2k = organizeImages(fileDir_j2k, labelsDir, '.j2k')


NIH_ir_bilinear_skimage = apply_model(model,
                clahe=False, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_jpg, fileNames=vars_jpg[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_NIH_ir_bilinear_skimage",
          var = NIH_ir_bilinear_skimage)
NIH_ir_interarea_cv2 = apply_model(model,
                    clahe=False, resize = 'interarea', eng = 'cv2', fileDirectory=fileDir_jpg, fileNames=vars_jpg[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_NIH_ir_interarea_cv2",
          var = NIH_ir_interarea_cv2)
NIH_ir_bilinear_cv2 = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'cv2', fileDirectory=fileDir_jpg, fileNames=vars_jpg[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_NIH_ir_bilinear_cv2",
          var = NIH_ir_bilinear_cv2)
NIH_ir_cubic_cv2 = apply_model(model,
                    clahe=False, resize = 'cubic', eng = 'cv2', fileDirectory=fileDir_jpg, fileNames=vars_jpg[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_NIH_ir_cubic_cv2",
          var = NIH_ir_cubic_cv2)
NIH_ir_lanczos_cv2 = apply_model(model,
                    clahe=False, resize = 'lanczos', eng = 'cv2', fileDirectory=fileDir_jpg, fileNames=vars_jpg[0])
pickleVar(path= contentFolder + "/AutoSavedVariables/"+modelName+"_NIH_ir_lanczos_cv2",
            var = NIH_ir_lanczos_cv2)

#Test CLAHE
NIH_clahe_true_bilinear_skimage = apply_model(model,
                      clahe=True, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_jpg, fileNames=vars_jpg[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_NIH_clahe_true_bilinear_skimage",
            var = NIH_clahe_true_bilinear_skimage)
#Test Compression
NIH_compr_j2k_bilinear_skimage = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_j2k, fileNames=vars_j2k[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_NIH_compr_j2k_bilinear_skimage",
          var = NIH_compr_j2k_bilinear_skimage)

#Compression by compression ratio

NIH_compr_j2k_cr2_bilinear_skimage = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_j2k_cr2, fileNames=vars_j2k[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_" + "NIH_compr_j2k_cr2_bilinear_skimage",
          var = NIH_compr_j2k_cr2_bilinear_skimage)
NIH_compr_j2k_cr4_bilinear_skimage = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_j2k_cr4, fileNames=vars_j2k[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_" + "NIH_compr_j2k_cr4_bilinear_skimage",
          var = NIH_compr_j2k_cr4_bilinear_skimage)
NIH_compr_j2k_cr8_bilinear_skimage = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_j2k_cr8, fileNames=vars_j2k[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_" + "NIH_compr_j2k_cr8_bilinear_skimage",
          var = NIH_compr_j2k_cr8_bilinear_skimage)
NIH_compr_j2k_cr16_bilinear_skimage = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_j2k_cr16, fileNames=vars_j2k[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_" + "NIH_compr_j2k_cr16_bilinear_skimage",
          var = NIH_compr_j2k_cr16_bilinear_skimage)
NIH_compr_j2k_cr32_bilinear_skimage = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_j2k_cr32, fileNames=vars_j2k[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_" + "NIH_compr_j2k_cr32_bilinear_skimage",
          var = NIH_compr_j2k_cr32_bilinear_skimage)
#TEST MIMIC DATA
#LOAD LABELS AND IMAGE NAMES
labelsDir = destinationFolder + 'mimic_cohort_png/labels.csv'
fileDir_jpg = destinationFolder + 'mimic_cohort_png/'
vars_jpg = organizeImages(fileDir_jpg, labelsDir, '.png')

fileDir_j2k = destinationFolder + 'mimic_j2k/'
fileDir_j2k_cr2 = destinationFolder + 'mimic_j2k_cr2/'
fileDir_j2k_cr4 = destinationFolder + 'mimic_j2k_cr4/'
fileDir_j2k_cr8 = destinationFolder + 'mimic_j2k_cr8/'
fileDir_j2k_cr16 = destinationFolder + 'mimic_j2k_cr16/'
fileDir_j2k_cr32 = destinationFolder + 'mimic_j2k_cr32/'


vars_j2k = organizeImages(fileDir_j2k, labelsDir, '.j2k')


mimic_ir_bilinear_skimage = apply_model(model,
                clahe=False, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_jpg, fileNames=vars_jpg[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_mimic_ir_bilinear_skimage",
          var = mimic_ir_bilinear_skimage)
mimic_ir_interarea_cv2 = apply_model(model,
                    clahe=False, resize = 'interarea', eng = 'cv2', fileDirectory=fileDir_jpg, fileNames=vars_jpg[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_mimic_ir_interarea_cv2",
          var = mimic_ir_interarea_cv2)
mimic_ir_bilinear_cv2 = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'cv2', fileDirectory=fileDir_jpg, fileNames=vars_jpg[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_mimic_ir_bilinear_cv2",
          var = mimic_ir_bilinear_cv2)
mimic_ir_cubic_cv2 = apply_model(model,
                    clahe=False, resize = 'cubic', eng = 'cv2', fileDirectory=fileDir_jpg, fileNames=vars_jpg[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_mimic_ir_cubic_cv2",
          var = mimic_ir_cubic_cv2)
mimic_ir_lanczos_cv2 = apply_model(model,
                    clahe=False, resize = 'lanczos', eng = 'cv2', fileDirectory=fileDir_jpg, fileNames=vars_jpg[0])
pickleVar(path= contentFolder + "/AutoSavedVariables/"+modelName+"_mimic_ir_lanczos_cv2",
            var = mimic_ir_lanczos_cv2)

#Test CLAHE
mimic_clahe_true_bilinear_skimage = apply_model(model,
                      clahe=True, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_jpg, fileNames=vars_jpg[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_mimic_clahe_true_bilinear_skimage",
            var = mimic_clahe_true_bilinear_skimage)
#Test Compression
mimic_compr_j2k_bilinear_skimage = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_j2k, fileNames=vars_j2k[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_mimic_compr_j2k_bilinear_skimage",
          var = mimic_compr_j2k_bilinear_skimage)

#Compression by compression ratio

mimic_compr_j2k_cr2_bilinear_skimage = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_j2k_cr2, fileNames=vars_j2k[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_" + "mimic_compr_j2k_cr2_bilinear_skimage",
          var = mimic_compr_j2k_cr2_bilinear_skimage)
mimic_compr_j2k_cr4_bilinear_skimage = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_j2k_cr4, fileNames=vars_j2k[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_" + "mimic_compr_j2k_cr4_bilinear_skimage",
          var = mimic_compr_j2k_cr4_bilinear_skimage)
mimic_compr_j2k_cr8_bilinear_skimage = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_j2k_cr8, fileNames=vars_j2k[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_" + "mimic_compr_j2k_cr8_bilinear_skimage",
          var = mimic_compr_j2k_cr8_bilinear_skimage)
mimic_compr_j2k_cr16_bilinear_skimage = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_j2k_cr16, fileNames=vars_j2k[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_" + "mimic_compr_j2k_cr16_bilinear_skimage",
          var = mimic_compr_j2k_cr16_bilinear_skimage)
mimic_compr_j2k_cr32_bilinear_skimage = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_j2k_cr32, fileNames=vars_j2k[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_" + "mimic_compr_j2k_cr32_bilinear_skimage",
          var = mimic_compr_j2k_cr32_bilinear_skimage)



In [ ]:
#TEST MIMIC DATA
#LOAD LABELS AND IMAGE NAMES
labelsDir = destinationFolder + 'mimic_cohort_png/labels.csv'
fileDir_jpg = destinationFolder + 'mimic_cohort_png/'
vars_jpg = organizeImages(fileDir_jpg, labelsDir, '.png')

fileDir_j2k = destinationFolder + 'mimic_j2k/'
fileDir_j2k_cr2 = destinationFolder + 'mimic_j2k_cr2/'
fileDir_j2k_cr4 = destinationFolder + 'mimic_j2k_cr4/'
fileDir_j2k_cr8 = destinationFolder + 'mimic_j2k_cr8/'
fileDir_j2k_cr16 = destinationFolder + 'mimic_j2k_cr16/'
fileDir_j2k_cr32 = destinationFolder + 'mimic_j2k_cr32/'


vars_j2k = organizeImages(fileDir_j2k, labelsDir, '.j2k')


#Compression by compression ratio

mimic_compr_j2k_cr2_bilinear_skimage = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_j2k_cr2, fileNames=vars_j2k[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_" + "mimic_compr_j2k_cr2_bilinear_skimage",
          var = mimic_compr_j2k_cr2_bilinear_skimage)
mimic_compr_j2k_cr4_bilinear_skimage = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_j2k_cr4, fileNames=vars_j2k[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_" + "mimic_compr_j2k_cr4_bilinear_skimage",
          var = mimic_compr_j2k_cr4_bilinear_skimage)
mimic_compr_j2k_cr8_bilinear_skimage = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_j2k_cr8, fileNames=vars_j2k[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_" + "mimic_compr_j2k_cr8_bilinear_skimage",
          var = mimic_compr_j2k_cr8_bilinear_skimage)
mimic_compr_j2k_cr16_bilinear_skimage = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_j2k_cr16, fileNames=vars_j2k[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_" + "mimic_compr_j2k_cr16_bilinear_skimage",
          var = mimic_compr_j2k_cr16_bilinear_skimage)
mimic_compr_j2k_cr32_bilinear_skimage = apply_model(model,
                    clahe=False, resize = 'bilinear', eng = 'skimage', fileDirectory=fileDir_j2k_cr32, fileNames=vars_j2k[0])
pickleVar(path=contentFolder + "/AutoSavedVariables/"+modelName+"_" + "mimic_compr_j2k_cr32_bilinear_skimage",
          var = mimic_compr_j2k_cr32_bilinear_skimage)


RESULTS PIPELINE

In [ ]:

#Models: "chexmodel" "xrv_nih" "mimic_ch"
#Datasets: "chex" "NIH" "mimic"

#models = ['chexmodel']
#datasets = ['chex']
#models = ['chexmodel', 'xrv_nih', 'xrv_mimic', 'xrv_chex']
#datasets = ['chex', 'mimic', 'NIH']
models = ['chexmodel', 'xrv_nih', 'xrv_mimic', 'xrv_chex']
datasets = ['chex', 'mimic', 'NIH']

#models = ['xrv_chex']
#datasets = ['chex']

res_names = ["res_bilinear_full", "res_Interarea", "res_BilinearCV2", "res_Cubic", "res_Lanczos", "res_CLAHE", "res_J2K", "res_2", "res_4", "res_8", "res_16", "res_32"]
mean_residual_groups_named = [x + '_avg_residual' for x in res_names]
perc_df_cols = ['BaselineCorrect',	'Consistently Correct',	'Consistently Wrong',
                'Correct Original, Wrong Transformed',	'Wrong Original, Correct Transformed',
                	'PercentFlipped',	'PreprocessedSensitivity',
                	'PreprocessedSpecificity',	'PreprocessedPPV',	'PreprocessedNPV',	'PreprocessedAccuracy',
    'PreprocessedAUROC']
count_df_cols = ['BaselineCorrect',	'Consistently Correct',	'Consistently Wrong',
                'Correct Original, Wrong Transformed',	'Wrong Original, Correct Transformed',
                	'PercentFlipped', 'BaselineSensitivity', 'BaselineSpecificity', 'PreprocessedSensitivity',
                'PreprocessedSpecificity', 'PreprocessedSensitivity_num',
                'PreprocessedSensitivity_denom', 'PreprocessedSpecificity_num',
                'PreprocessedSpecificity_denom', 'PreprocessedPPV_num',
                'PreprocessedPPV_denom', 'PreprocessedNPV_num', 'PreprocessedNPV_denom',
                'PreprocessedAccuracy_num', 'PreprocessedAccuracy_denom']

tstat_df = pd.DataFrame(columns = ['model', 'dataset', 'name', 'T_statistic', 'p_value'])

tscatter_df = pd.DataFrame()
tdeployed_list = []
tcounts_list = []

mean_residuals_sorted_list = []

#Dataframe to hold all results data together
columns_per_repetition_perc = len(perc_df_cols)
columns_per_repetition_count = len(count_df_cols)


# Define the prefixes for each repetition
prefixes = res_names

# Create a list of all column names (including the additional columns)
all_columns_perc = ['model', 'dataset', 'disease_label'] + [f"{prefix}_{perc_df_cols[i]}" for prefix in prefixes for i in range(0, columns_per_repetition_perc)] + ['residual_order'] + mean_residual_groups_named
all_columns_count = ['model', 'dataset', 'disease_label'] + [f"{prefix}_{count_df_cols[i]}" for prefix in prefixes for i in range(0, columns_per_repetition_count)]
#Dfs for individually stratified results
continuous_percent_df = pd.DataFrame(columns=all_columns_perc)
continuous_counts_df = pd.DataFrame(columns=all_columns_count)

for m in models:
  if m == 'chexmodel':
      pathologies = chex_pathologies
  else:
      pathologies = included_pathologies

  for d in datasets:
    mdscatter_df = pd.DataFrame()

    if d == 'chex':
      #TEST CHEX DATA
      labelsDir = destinationFolder + 'chex_cohort/labels.csv'
      fileDir_jpg = destinationFolder + 'chex_cohort/'
      vars_jpg = organizeImages(fileDir_jpg, labelsDir, '.jpg')
    elif d == 'NIH':
      #TEST NIH DATA
      labelsDir = destinationFolder + 'NIH_cohort_balanced/labels.csv'
      fileDir_jpg = destinationFolder + 'NIH_cohort_balanced/'
      vars_jpg = organizeImages(fileDir_jpg, labelsDir, '.png')

    elif d == 'mimic':
      #TEST MIMIC DATA
      labelsDir = destinationFolder + 'mimic_cohort_png/labels.csv'
      fileDir_jpg = destinationFolder + 'mimic_cohort_png/'
      vars_jpg = organizeImages(fileDir_jpg, labelsDir, '.png')
    else:
      raise ValueError('Dataset name not recognized. Please use either "chex", "NIH", or "mimic".')

    savedVariables = loadSavedVariables(pathologies, m, d, preprocessingMethods)
    #If it is the NIH model and we want to exclude the points coded as RGB
    if d == "NIH":
      #Need RGB_indices defined
      #fixing savedvariables[1]
      for x in pathologies:
        savedVariables[1][x] = savedVariables[1][x].drop(pd.Series(RGB_indices)).reset_index(drop=True)
      #fixing savedvariables[0]
      for x in range(len(savedVariables[0])):

        for i in sorted(RGB_indices, reverse=True):
          del savedVariables[0][x][i]
      #Also fix vars_jpg
      tmp_list = list(vars_jpg[1])
      for i in sorted(RGB_indices, reverse=True):
          del tmp_list[i]

      tuple_list = list(vars_jpg)
      # update item
      tuple_list[1] = tmp_list
      # convert list back to tuple
      vars_jpg = tuple(tuple_list)

    pa = getStatisticalResults(pathologies, savedVariables, vars_jpg)
    pa.insert(0, 'dataset', d)
    pa.insert(0, 'model', m)
    tstat_df = pd.concat([tstat_df, pa], ignore_index=True, axis=0)


    #This section is if we need to regenerate spreadsheets for a deployed model's performance
    percent_df_list = []
    count_df_list = []
    regenerateSpreadsheets = True

    if regenerateSpreadsheets:
      for c in range(len(pathologies)):
          #Skip consolidation since it's in chex model pathologies but we aren't using it
          if (m == 'chexmodel') and c==2:
            #no nothing
            pass
          else:
            condition_df, points_df = get_predictions_and_scatterpoints(m, d, c, doCLAHEandJ2K = True, doCompRatio = True, savedVariables = savedVariables)

            #Can calculate mean residuals here
            #print(points_df.columns)

            mean_residuals = []
            mean_residual_groups = []
            for x in range(0, len(points_df.columns), 2):


              # Calculate the residuals from the line y = x
              #print(points_df.columns[x+1])
              #print(points_df.columns[x])
              residuals = np.array(points_df[points_df.columns[x+1]]) - np.array(points_df[points_df.columns[x]])
              abs_residuals = np.abs(residuals)

              # Calculate the mean residual
              mean_residual = np.mean(abs_residuals)

              #print("Mean Residual:")
              #print(points_df.columns[x])
              #print(mean_residual)



              mean_residuals.append(mean_residual)
              mean_residual_groups.append(points_df.columns[x])

            # Sort the mean_residuals and mean_residual_groups together based on mean_residuals
            sorted_mean_residuals, sorted_mean_residual_groups = zip(*sorted(zip(mean_residuals, mean_residual_groups)))

            # Print mean_residual_groups in the sorted order
            #print("Sorted Mean Residual Groups:")
            #print(sorted_mean_residual_groups)
            mean_residuals_sorted_list.append(sorted_mean_residual_groups)

            mdscatter_df = pd.concat([mdscatter_df, points_df], ignore_index=True)

            conditionThreshold = getYoudensThreshold(condition_df[f'y_test_{pathologies[c]}_Bilinear'], condition_df[f'preds_{pathologies[c]}_Bilinear'])

            percent_df, count_df = getCounts(condition_df, conditionThreshold)
            if '' in count_df.columns:
              count_df = count_df.drop('', axis=1)

            #Adding to continuous_percent_df
            percent_df_concat_list = percent_df.iloc[:, :len(perc_df_cols)].to_numpy().flatten().tolist()
            new_row_perc_df = [m, d, pathologies[c]] + percent_df_concat_list + [sorted_mean_residual_groups] + mean_residuals

            #Adding to continous_counts_df
            counts_df_concat_list = count_df.iloc[:, :len(count_df_cols)].to_numpy().flatten().tolist()
            new_row_count_df = [m, d, pathologies[c]] + counts_df_concat_list

            #0 new_row_perc_df = [m, d, pathologies[c]] + percent_df_concat_list + [sorted_mean_residual_groups] + mean_residuals


            print([m, d, pathologies[c]])

            insert_index = len(continuous_percent_df)

            # Insert the new row into continuous_percent_df
            print(len(new_row_count_df))
            print(len(all_columns_count))
            continuous_percent_df.loc[insert_index] = new_row_perc_df
            continuous_counts_df.loc[insert_index] = new_row_count_df

            percent_df_list.append(percent_df)
            count_df_list.append(count_df)

          comb_percent_df = pd.concat(percent_df_list).groupby(level=0).mean()
          comb_percent_df.to_csv(f"/content/{m}_{d}_averaged.csv")

          comb_count_df = sum(count_df_list)
          comb_count_df.to_csv(f"/content/{m}_{d}_counts.csv")



    tscatter_df = pd.concat([tscatter_df, mdscatter_df], ignore_index=True)
    tdeployed_list.append(comb_percent_df)
    tcounts_list.append(comb_count_df)



#tscatter_df is all set
#tdeployed_list now needs to be averaged into a single dataframe
tdeployed_df = pd.concat(tdeployed_list).groupby(level=0).mean()
#tcounts_list needs to be summed into a single dataframe
tcounts_df = sum(tcounts_list)
#Write to csvs
#tscatter contains the prediction outputs normalized
tscatter_df.to_csv("/content/tscatter_df.csv")
#tdeployed contains the average values of correct/incorrect/sensitivity/specificity etc.
tdeployed_df.to_csv("/content/tdeployed_df.csv")
#tcounts is used for chi squared tests of counts
tcounts_df.to_csv("/content/tcounts_df.csv")
#continuous_percent_df contains percent_df which becomes tdeployed except
#retains the individual results for each model x dataset x disease label
continuous_percent_df.to_csv("/content/continuous_percent_df.csv")

continuous_counts_df.to_csv("/content/continuous_counts_df.csv")
